In [1]:
#Same as V3 , but updated so it reports Subset numbers
#Same as V4 but Subsets count are updated as the flowchart
#gtv00_listPx is not used now

In [34]:
from os.path import join
from datetime import time, datetime
import os
import glob
import numpy as np
from skimage.draw import polygon
import SimpleITK as sitk
#import pydicom as dicom
#from pydicom.tag import Tag
import sys
import csv
import pandas as pd
import shutil

In [39]:
import yaml
with open('config_CheckNiiOk.yaml', 'r') as stream:
    config = yaml.safe_load(stream)
    
    itvtot = config['itv_tot_labels']
    itvtum = config['itv_tumor_labels']
    itvnod = config['itv_ln_labels']

    
    gtvtot = config['gtv_tot_labels']
    gtvtum = config['gtv_tumor_labels']
    gtvnod = config['gtv_ln_labels']
    gtv00 = config['gtv_expi00_labels']

    rootp_Nii = config['root_Nii']

    aveTags = config['aveTags']
    bpTags =  config['bPTags']

    savecsv_path = config['save_csv']
    fieldnames = config['fieldnames']

with open('//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/ListsOfPatients/PxSelectionforDCM2Nii.txt', 'r') as file:
    PxList = [line.rstrip() for line in file]
    
#PxList = os.listdir(rootp_Nii)
print(len(PxList),'-',PxList)
target_labels = itvtot + itvtum+itvnod+gtvtot+gtvtum+gtvnod

861 - ['4291861', '3327503', '4693524', '6184000', '6163057', '659641', '1933846', '2083343', '1616581', '2007559', '8034617', '6959090', '2621487', '2345065', '7431977', '7786608', '7846169', '8962420', '8192437', '4156751', '1780721', '6224291', '4112868', '7153622', '5735628', '5267139', '1213955', '2628217', '8510422', '8698891', '7706205', '1187542', '8725573', '8715455', '1127507', '3980744', '4023673', '9212857', '8751725', '4709744', '9105082', '4342472', '8728753', '1397990', '6926232', '9588158', '3059294', '8778014', '4575927', '4809024', '2531074', '4960069', '772611', '6557018', '3307795', '3379144', '772475', '1663955', '8884976', '885235', '8882956', '8874736', '3082668', '3443633', '3968754', '4927095', '3851734', '8726194', '986614', '6402783', '5474853', '2047597', '8223634', '9436340', '8156441', '5426768', '340941', '3717936', '4996487', '9005599', '4117321', '2788596', '4032982', '2835183', '9034825', '4239862', '9025583', '9061277', '5938385', '7313332', '2162911'

In [40]:
gtv00

['GTVtumor_insp']

In [41]:
subsetA11_count =0
subsetA21_count =0
subsetA22_count =0
subsetB11_count =0
subsetB21_count =0
subsetE1_count =0
subsetE2_count=0
subsetOff_count=0

missingGTV = 0
missingITV=0
missingPlanCT =0
missing2BP =0
missing10BP =0 
corruptedPx_dict = {}
corruptedPx_dict2fix = {}
gtv00_listPx = []
missingDelineation =0


with open(savecsv_path, mode='w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    for patientID in PxList:
        itvtot_b,itvtum_b,itvnod_b=(False,False,False)
        gtvtot_b,gtvtum_b,gtvnod_b=(False,False,False)
        BP0_b,BP10_b,BP20_b,BP30_b,BP40_b,BP50_b,BP60_b,BP70_b,BP80_b,BP90_b,BP100_b=(False,False,False,False,False,False,False,False,False,False,False)
        planCT_b = False
        check_b = False

        for root, dirs, files in os.walk(os.path.join(rootp_Nii,patientID), topdown=True):
            for f in files: 
                #Average
                if any(planct_tt in f.lower() for planct_tt in aveTags):
                    planCT_b = True
                #ITV
                if  any(itvtot_tt in f for itvtot_tt in itvtot):
                    itvtot_b = True
                if any(itvtum_tt in f for itvtum_tt in itvtum):
                    itvtum_b = True
                if any(itvnod_tt in f for itvnod_tt in itvnod):
                    itvnod_b = True
                #GTV
                if any(gtvtot_tt in f for gtvtot_tt in gtvtot):
                    gtvtot_b = True
                if any(gtvtum_tt in f for gtvtum_tt in gtvtum):
                    gtvtum_b = True
                if any(gtvnod_tt in f for gtvnod_tt in gtvnod):
                    gtvnod_b = True
                if any(gtv00_tt in f for gtv00_tt in gtv00):
                    gtv00_listPx.append(patientID)
                #BP
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["0%"]):
                    BP0_b =True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["10%"]):
                    BP10_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["20%"]):
                    BP20_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["30%"]):
                    BP30_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["40%"]):
                    BP40_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["50%"]):
                    BP50_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["60%"]):
                    BP60_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["70%"]):
                    BP70_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["80%"]):
                    BP80_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["90%"]):
                    BP90_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["100%"]):
                    BP100_b = True
    
        if planCT_b and (itvtot_b or itvtum_b or itvnod_b) and (gtvtot_b or gtvtum_b or gtvnod_b) and (BP50_b or BP60_b) and (BP0_b,BP100_b,BP90_b):
            check_b = True
        #Subset SUM    
        if (itvtot_b or itvtum_b or itvnod_b) == False:
            subsetE1_count+=1
            corruptedPx_dict2fix[patientID] = "ITV Missing"
        elif (BP40_b or BP50_b or BP60_b) == False:
            subsetE2_count+=1
            corruptedPx_dict2fix[patientID] = "No Max Exp BP"
        elif (BP0_b or BP100_b or BP90_b or BP10_b) and (BP40_b or BP50_b or BP60_b) and (BP20_b or BP80_b) and (BP30_b or BP70_b):
            subsetA11_count+=1
            if (gtvtot_b or gtvtum_b or gtvnod_b):
                subsetA21_count+=1
            else:
                subsetA22_count+=1
        elif (BP40_b or BP50_b or BP60_b) == True:
            subsetB11_count+=1
            if (gtvtot_b or gtvtum_b or gtvnod_b):
                subsetB21_count+=1
            else:
                subsetOff_count+=1
        
        #ERRORS sum
        if not(itvtot_b or itvtum_b or itvnod_b):
            missingITV +=1
            corruptedPx_dict[patientID] = "MissingITV"
        if not(gtvtot_b or gtvtum_b or gtvnod_b):
            missingGTV +=1
            corruptedPx_dict[patientID] = "MissingGTV"
        if not((BP50_b or BP60_b) and (BP0_b,BP100_b,BP10_b,BP90_b)):
            missing2BP +=1
            corruptedPx_dict[patientID] = "Missing Peak Phases"
            #print(files)
        if not(planCT_b):
            missingPlanCT +=1
            corruptedPx_dict[patientID] = "Missing Plan CT"
        writer.writerow({
            "PatientID": patientID,"PlanCT": planCT_b,
            "ITV_Total": itvtot_b,"ITV_Tumor": itvtum_b,"ITV_Nodal": itvnod_b,
            "GTV_Total": gtvtot_b,"GTV_Tumor": gtvtum_b,"GTV_Nodal": gtvnod_b,
            "BP_0%": BP0_b,"BP_10%": BP10_b,"BP_90%": BP90_b,"BP_100%": BP100_b,
            "BP_20%": BP20_b,"BP_30%": BP30_b,
            "BP_40%": BP40_b,"BP_50%": BP50_b,"BP_60%": BP60_b,
            "BP_70%": BP70_b,"BP_80%": BP80_b,"Check":check_b
        })


In [42]:
print("E1:",subsetE1_count,"E2: ",subsetE2_count)
print("B11:",subsetB11_count,"A11: ",subsetA11_count)
print("B21:",subsetB21_count,"A21: ",subsetA21_count,"A22: ",subsetA22_count)

TotOkPx = subsetB21_count+subsetA21_count+subsetA22_count
Px2Train = np.round(TotOkPx*.8).astype(np.uint64)
Px2Test =  np.round(TotOkPx*.2).astype(np.uint64)


print("Patients to Train:",Px2Train,"Test",Px2Test)
print("Train Sum",subsetB21_count,Px2Train-subsetB21_count)
print("Test Sum",subsetA22_count,subsetA21_count-(Px2Train-subsetB21_count))
print("Tot ok Px",TotOkPx)
print("SubsetOff",subsetOff_count)

E1: 16 E2:  3
B11: 167 A11:  675
B21: 162 A21:  662 A22:  13
Patients to Train: 670 Test 167
Train Sum 162 508.0
Test Sum 13 154.0
Tot ok Px 837
SubsetOff 5


In [43]:
len(corruptedPx_dict2fix)

19

In [44]:
corruptedPx_dict2fix

{'1213955': 'ITV Missing',
 '8725573': 'No Max Exp BP',
 '4927095': 'No Max Exp BP',
 '4239862': 'ITV Missing',
 '6934596': 'ITV Missing',
 '7901414': 'ITV Missing',
 '5609966': 'ITV Missing',
 '1539651': 'ITV Missing',
 '9339426': 'ITV Missing',
 '2375717': 'No Max Exp BP',
 '1136033': 'ITV Missing',
 '9549714': 'ITV Missing',
 '892781': 'ITV Missing',
 '7210726': 'ITV Missing',
 '13194': 'ITV Missing',
 '4965499': 'ITV Missing',
 '5317512': 'ITV Missing',
 '2334091': 'ITV Missing',
 '100789': 'ITV Missing'}

In [14]:
print("Missing Delineation",missingGTV,missingITV,"BP",missing2BP,"PlanCT",missingPlanCT)


Missing Delineation 26 24 BP 132 PlanCT 5


In [7]:
len(corruptedPx_dict)

169

In [8]:
corruptedPx_dict

{'2096249': 'MissingGTV',
 '7236265': 'Missing Peak Phases',
 '7513114': 'Missing Peak Phases',
 '766115': 'Missing Peak Phases',
 '23840': 'Missing Peak Phases',
 '4023673': 'Missing Peak Phases',
 '2856145': 'Missing Peak Phases',
 '5196470': 'Missing Peak Phases',
 '5317512': 'MissingGTV',
 '5635761': 'Missing Peak Phases',
 '8716256': 'MissingGTV',
 '3741290': 'Missing Peak Phases',
 '5666561': 'Missing Peak Phases',
 '367466': 'Missing Peak Phases',
 '8036407': 'Missing Peak Phases',
 '5889191': 'Missing Plan CT',
 '32628': 'Missing Peak Phases',
 '3853415': 'Missing Peak Phases',
 '2084312': 'MissingGTV',
 '2758081': 'Missing Peak Phases',
 '9193538': 'Missing Peak Phases',
 '9918855': 'MissingGTV',
 '654137': 'Missing Peak Phases',
 '13194': 'MissingGTV',
 '5468590': 'Missing Peak Phases',
 '1136033': 'MissingGTV',
 '4965499': 'MissingGTV',
 '2334091': 'MissingITV',
 '2370282': 'Missing Peak Phases',
 '9854449': 'Missing Peak Phases',
 '9646447': 'Missing Peak Phases',
 '246826'